# Robot Demo

This demo proves **motors** work via **direct motor control** (`/ros_robot_controller/set_motor_speeds`).

- Movement (forward/back/turn/strafe/diagonal)
- Drift (wheel-level)
- Horn (meep meep)

Camera + TTS are separate and can stay as-is for now.

In [ ]:
# --- Bootstrap: add common/lib and lessons/lib to sys.path ---
from pathlib import Path
import sys

def find_repo_root() -> Path:
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'common').is_dir() and (p / 'lessons').is_dir():
            return p
    raise FileNotFoundError('Could not find repo root (needs common/ and lessons/)')

ROOT = find_repo_root()
COMMON_LIB = ROOT / 'common' / 'lib'
LESSONS_LIB = ROOT / 'lessons' / 'lib'

for path in (COMMON_LIB, LESSONS_LIB):
    if str(path) not in sys.path:
        sys.path.insert(0, str(path))

print('Repo root:', ROOT)
print('Using common lib:', COMMON_LIB)
print('Python:', sys.version.split()[0])

In [ ]:
import time, importlib
import robot_moves as rm
importlib.reload(rm)

bot = rm.RobotMoves(base_speed=220, rate_hz=30)
print('RobotMoves ready. BASE_SPEED=', rm.BASE_SPEED, 'RATE_HZ=', rm.RATE_HZ)

In [ ]:
# Quick safety: stop first
bot.stop()
time.sleep(0.2)
print('Stopped.')

In [ ]:
print('Forward / Backward')
bot.forward(0.6)
time.sleep(0.2)
bot.backward(0.6)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Turn left / right')
bot.turn_left(0.5)
time.sleep(0.2)
bot.turn_right(0.5)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Strafe left / right')
bot.left(0.6)
time.sleep(0.2)
bot.right(0.6)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Diagonals')
bot.diagonal_left(0.6)
time.sleep(0.2)
bot.diagonal_right(0.6)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Drift demo (wheel-level, no /cmd_vel)')
bot.drift_left(seconds=0.9, turn_blend=0.55)
time.sleep(0.2)
bot.drift_right(seconds=0.9, turn_blend=0.55)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Horn: meep meep')
bot.horn(block=True)
print('Done')